## LGBM gpu install

In [1]:
!git clone https://github.com/Microsoft/LightGBM

%cd LightGBM
!mkdir build
!cmake -DUSE_GPU=1
!make -j$(nproc)
!sudo apt-get -y install python-pip
!sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
%cd /content/LightGBM/python-package
!sudo python setup.py install
!pip3 uninstall scikit-learn
!pip3 install scikit-learn==0.21.3





fatal: destination path 'LightGBM' already exists and is not an empty directory.
/content/LightGBM
mkdir: cannot create directory ‘build’: File exists
-- Found OpenMP_C: -fopenmp  
-- Found OpenMP_CXX: -fopenmp  
-- Found OpenMP: TRUE   
-- OpenCL include directory: /usr/include
-- Boost version: 1.65.1
-- Found the following Boost libraries:
--   filesystem
--   system
-- Using _mm_prefetch
-- Using _mm_malloc
-- Configuring done
-- Generating done
-- Build files have been written to: /content/LightGBM
[ 95%] Built target lightgbm
[100%] Built target _lightgbm
Reading package lists... Done
Building dependency tree       
Reading state information... Done
python-pip is already the newest version (9.0.1-2.3~ubuntu1.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Requirement already up-to-date: setuptools in /usr/local/lib/python3.6/dist-packages (50.3.0)
Requirement already up-to-date: pandas in /usr/local/lib/python3.6/dist-packages (1.1.2)
Requirement already

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [81]:
import lightgbm as lgb
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
import pandas as pd
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler # 데이터 표준화

import seaborn as sns
import warnings
import math
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import log_loss

# lightgbm 모델
from lightgbm import LGBMClassifier, plot_importance
import lightgbm as lgb
# Xgboost 모델
import xgboost as xgb

# 랜덤포레스트 
from sklearn.ensemble import RandomForestClassifier

# 로지스틱 리그레션
from sklearn.linear_model import LogisticRegression

# normalization
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

#feature Selection
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.feature_selection import f_classif
import itertools

from sklearn.decomposition import PCA
from sklearn.decomposition import IncrementalPCA
from sklearn.decomposition import KernelPCA

from google.colab import files

In [5]:
from tensorflow.keras import models
from tensorflow.keras import layers
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import *
from sklearn.preprocessing import *
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [6]:
!pip install optuna
import optuna

In [7]:
rcParams['figure.figsize'] = (18, 9)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [8]:
!cp /content/drive/My\ Drive/Daycon/input/fg_trn.csv /content/
!cp /content/drive/My\ Drive/Daycon/input/fg_tst.csv /content/
!cp /content/drive/My\ Drive/Daycon/input/sample_submission.csv /content/

In [9]:

trn_file = '/content/fg_trn.csv'
tst_file = '/content/fg_tst.csv'
# feature_file = data_dir / 'feature.csv'
sample_file = '/content/sample_submission.csv'

target_col = 'class'
seed = 42

In [10]:
def cross_entropy(p, q):
	return -sum([p[i]*log2(q[i]) for i in range(len(p))])

In [11]:
trn = pd.read_csv(trn_file,index_col=0)
print(trn.shape)
trn.head()

(318740, 91)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,class
0,23.2640,20.3368,19.0095,17.6724,16.9396,-8.1086e-05,17.6321,2.9272,23.2640,3.0062,1.3273,2.6643,-2.7875,1.3817,1.3371,2.0699,19.0096,-4.1148,-1.2483,1.3774,17.6725,-2.5854,-1.2827,-2.0155,-0.6925,0.0307,-17.6322,-16.9090,2.8665,4.1691,1.3027,2.6257,1.3230,2.0462,0.7232,1.1439,-2.8690e+05,1.0698,-2.5080e+05,0.8795,1.0729,1.0757,0.8221,0.9384,1.0433,-2.1795e+05,0.8724,0.9323,-2.0891e+05,0.8937,1.0018,-3.5065e-06,-4.0027e-06,-4.2778e-06,-4.5988e-06,-4.7955e-06,-4.5048e-06,-4.5048e-06,-6.8153e-05,-6.8101e-05,-6.8177e-05,-6.8128e-05,1.1415,1.2199,1.0687,1.1489,1.0750,1.1210,1.0428,4.1012e+05,-44588.1645,700.0365,-705.3298,-2043.2987,6.3963,-1.8975,0.1039,-1.1943,-0.1168,-0.3289,-1.2392,0.8164,23.2640,6.3243,6.2153,2.4562,0.0017,0.0307,91.2733,21.9336,0.0
1,15.0521,14.0620,13.4524,13.2684,13.1689,4.5061e-03,13.2363,0.9902,15.0476,1.0476,0.6096,0.7935,-0.9045,0.6506,0.1840,0.2835,13.4479,-1.5140,-0.5521,0.2162,13.2639,-0.7361,-0.1429,-0.2424,-0.0673,0.0343,-13.2317,-13.1302,0.9619,1.5550,0.5931,0.7683,0.1751,0.2767,0.1016,1.0704,3.3404e+03,1.0453,3.1207e+03,0.9396,1.0485,1.0139,0.8988,0.9606,1.0076,2.9446e+03,0.9474,0.9893,2.9225e+03,0.9819,1.0026,3.0108e-04,3.2176e-04,3.3599e-04,3.4043e-04,3.4307e-04,4.5061e-03,4.5061e-03,3.5955e-03,3.5824e-03,3.6020e-03,3.5890e-03,1.0687,1.1159,1.0442,1.0580,1.0132,1.0211,1.0077,-1.3453e+05,1329.3993,234.1073,-139.3329,141.9778,6.1358,0.9277,-2.3727,0.1881,-0.6354,-0.4376,1.1678,-0.0957,15.0521,1.8832,1.8317,0.7586,0.0091,0.0343,62.7375,13.7086,1.0
2,16.7864,15.8254,15.5363,15.3935,15.3500,4.7198e-04,15.3217,0.9610,16.7859,1.0998,0.2891,0.4319,-0.7822,0.3854,0.1428,0.1863,15.5358,-1.0713,-0.1503,0.2146,15.3931,-0.2931,-0.0465,-0.0900,0.0283,0.0540,-15.3213,-15.2956,0.9211,1.1676,0.2466,0.3649,0.1183,0.1440,0.0257,1.0607,3.5566e+04,1.0186,3.3530e+04,0.9529,1.0250,1.0093,0.9355,0.9904,1.0028,3.2615e+04,0.9813,0.9970,3.2523e+04,0.9942,1.0035,2.8419e-05,3.0088e-05,3.0568e-05,3.0804e-05,3.0856e-05,2.3599e-04,2.3599e-04,4.6159e-04,4.6087e-04,4.6194e-04,4.6123e-04,1.0587,1.0756,1.0160,1.0238,1.0077,1.0094,1.0017,-2.0247e+05,1399.9650,333.2776,117.0187,206.6743,8.5752,0.3751,-0.9788,0.0770,-0.7371,0.0022,0.2375,0.3560,16.7864,1.4364,1.3116,0.5462,0.0031,0.0540,73.7791,15.5835,0.0
3,25.6606,21.1887,20.2212,19.8949,19.6346,5.8143e-06,19.7947,4.4719,25.6606,4.6660,0.9676,1.2938,-4.1649,1.1015,0.3262,0.5865,20.2212,-5.1325,-0.7735,0.4265,19.8949,-1.0997,-0.1923,-0.4526,-0.1600,0.0795,-19.7946,-19.5552,4.3590,5.2664,0.9074,1.2000,0.2926,0.5321,0.2395,1.2111,4.4134e+06,1.0478,3.6442e+06,0.8357,1.0548,1.0164,0.7976,0.9632,1.0133,3.4217e+06,0.9476,0.9904,3.3770e+06,0.9775,1.0041,2.2933e-07,2.7694e-07,2.8945e-07,2.9373e-07,2.9733e-07,1.4536e-06,1.9381e-06,4.8235e-06,4.8209e-06,4.8247e-06,4.8223e-06,1.2076,1.2622,1.0452,1.0606,1.0148,1.0272,1.0122,-8.3498e+06,540693.7038,-255606.5334,48177.6402,58140.3174,-3.8705,4.3914,10.1283,-5.9023,0.3165,-0.6332,-3.9224,-0.1401,25.6606,6.0260,5.7985,2.4085,0.0012,0.0795,100.5730,24.1476,0.0
4,24.4534,20.6992,19.0424,18.3242,17.9826,-3.3247e-05,18.1903,3.7543,24.4535,4.0196,1.6568,2.3750,-3.0722,1.8362,0.7182,1.0597,19.0424,-4.7290,-1.3915,0.8521,18.3242,-2.1097,-0.5388,-0.8803,-0.2076,0.1067,-18.1903,-17.8760,3.3376,4.9084,1.5709,2.2436,0.6727,0.9871,0.3144,1.1814,-7.3551e+05,1.0870,-6.2259e+05,0.8708,1.0973,1.0392,0.8011,0.9319,1.0190,-5.5115e+05,0.8968,0.9714,-5.4088e+05,0.9533,1.0060,-1.3986e-06,-1.6271e-06,-1.7626e-06,-1.8277e-06,-1.8599e-06,-2.5575e-06,-2.7706e-06,-2.7846e-05,-2.7838e-05,-2.7849e-05,-2.7843e-05,1.1633,1.2602,1.0833,1.1233,1.0370,1.0552,1.0176,1.2285e+06,-112285.4841,28776.9020,2431.2498,-1802.2420,3.5764,2.3899,5.9035,-2.4057,0.1657,1.1459,-0.3625,-0.5340,24.4534,6.4708,5.8955,2.4156,0.0005,0

In [12]:
tst = pd.read_csv(tst_file, index_col = 0)
tst

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89
0,19.1431,18.0172,17.4733,17.1144,16.9263,1.2130e-01,17.0838,1.1259,19.0217,1.1881,0.5439,0.9028,-1.0578,0.5863,0.3588,0.5470,17.3520,-1.6017,-0.4817,0.3894,16.9931,-0.8405,-0.3164,-0.5046,-0.1575,0.0176,-16.9625,-16.7874,1.1200,1.6441,0.5241,0.8711,0.3470,0.5221,0.1751,1.0625,157.8113,1.0311,148.5300,0.9445,1.0336,1.0210,0.9160,0.9732,1.0111,141.0876,0.9532,0.9818,139.5368,0.9711,1.0010,6.3593e-03,6.7560e-03,6.9591e-03,7.1005e-03,7.1740e-03,1.2130e-01,1.2130e-01,1.0490e-01,1.0535e-01,1.0468e-01,1.0512e-01,1.0624,1.0943,1.0301,1.0510,1.0203,1.0309,1.0104,-1.2800e+05,1196.9230,235.6550,-139.1880,160.5277,-1.1646,-0.3764,-0.6646,-0.3363,0.2974,0.2247,0.1788,0.0961,19.1431,2.2168,2.1662,0.8708,9.8340e-03,0.0176,82.8923,17.9136
1,18.7409,17.1705,16.3691,15.9865,15.7307,-6.6907e-05,14.5160,1.5704,18.7410,4.4312,0.8014,1.1840,2.1001,2.7804,0.3826,0.6384,16.3692,1.2988,2.0593,1.8531,15.9865,1.6767,1.5963,1.3406,1.2147,1.0935,-14.5161,-14.6372,0.7606,0.6802,-0.0803,-0.2062,-0.1259,-0.2470,-0.1212,1.0915,-280103.3378,1.0490,-256631.4466,1.1394,1.1932,1.0239,1.0862,1.1439,1.0163,-238934.9505,1.1172,1.1109,-235112.5900,1.0932,1.0747,-4.4397e-06,-4.6756e-06,-4.6495e-06,-4.6092e-06,-4.5710e-06,-3.3454e-05,-3.3454e-05,-6.6400e-05,-6.6417e-05,-6.6390e-05,-6.6409e-05,1.0532,1.0473,0.9944,0.9858,0.9913,0.9831,0.9917,4.3461e+05,-18062.1118,1116.7790,-865.0933,751.4398,11.0658,2.4369,7.5851,14.9594,-0.4639,0.4538,-1.2207,-0.4063,18.7409,3.0102,0.7606,0.2986,5.9200e-04,1.4209,78.9593,14.0624
2,19.2980,17.8803,17.2221,16.8096,16.6352,1.1001e-01,16.7497,1.4177,19.1880,1.5254,0.6582,1.0708,-1.2200,0.7343,0.4126,0.5869,17.1121,-1.8782,-0.5505,0.4725,16.6996,-0.9631,-0.3364,-0.5108,-0.1144,0.0664,-16.6397,-16.4589,1.3277,1.9543,0.6266,1.0230,0.3963,0.5771,0.1808,1.0793,175.4203,1.0382,162.5333,0.9361,1.0428,1.0245,0.9017,0.9690,1.0105,152.7998,0.9458,0.9804,151.2152,0.9702,1.0040,5.7596e-03,6.1899e-03,6.4161e-03,6.5679e-03,6.6396e-03,1.1001e-01,1.1001e-01,7.9668e-02,7.9116e-02,7.9941e-02,7.9393e-02,1.0747,1.1140,1.0365,1.0611,1.0237,1.0348,1.0109,-1.2803e+05,1200.7489,234.4996,-139.4618,160.7233,-7.9098,-0.2344,0.1504,-0.0154,0.5051,0.2173,0.1883,0.1117,19.2980,2.6628,2.5315,1.0229,1.8988e-02,0.0664,80.9407,17.7099
3,18.0775,17.1569,16.9041,16.8118,16.7955,2.9697e-05,16.7700,0.9205,18.0774,1.0039,0.2528,0.3451,-0.8160,0.3100,0.0923,0.1086,16.9041,-1.0688,-0.1694,0.1342,16.8118,-0.2617,-0.0352,-0.0514,0.0256,0.0288,-16.7699,-16.7667,0.8994,1.1260,0.2266,0.3036,0.0770,0.0802,0.0032,1.0537,608734.6509,1.0150,577737.2243,0.9546,1.0184,1.0055,0.9405,0.9901,1.0010,566115.0006,0.9847,0.9979,565566.8063,0.9969,1.0017,1.6523e-06,1.7393e-06,1.7627e-06,1.7708e-06,1.7712e-06,2.9697e-05,2.9697e-05,2.6032e-05,2.6031e-05,2.6031e-05,2.6032e-05,1.0527,1.0668,1.0134,1.0181,1.0046,1.0048,1.0002,-1.4194e+06,-2626.0413,2741.7742,4617.5482,598.5527,-1.5521,-0.2674,-0.0394,-0.3483,-0.3142,0.2022,0.0634,0.1874,18.0775,1.2819,1.2062,0.5113,1.1600e-04,0.0288,80.0417,16.8320
4,17.8325,16.7646,16.2135,15.8722,15.6621,6.8471e-02,15.8315,1.0679,17.7640,1.1530,0.5511,0.8924,-0.9899,0.6083,0.3413,0.5514,16.1451,-1.5409,-0.4659,0.3821,15.8037,-0.8072,-0.2841,-0.4942,-0.1694,0.0187,-15.7630,-15.5749,1.0750,1.5982,0.5232,0.8480,0.3248,0.5129,0.1881,1.0637,260.4378,1.0340,244.8417,0.9442,1.0377,1.0215,0.9132,0.9721,1.0134,231.8084,0.9516,0.9824,228.7400,0.9694,1.0012,3.8566e-03,4.1051e-03,4.2381e-03,4.3250e-03,4.3770e-03,3.4236e-02,3.4236e-02,6.1629e-02,6.1816e-02,6.1533e-02,6.1723e-02,1.0645,1.0989,1.0324,1.0536,1.0205,1.0328,1.0120,-1.2821e+05,1201.4544,236.6646,-138.7002,160.2035,4.5367,0.1147,-1.5739,-0.0037,-0.2064,0.0311,0.3848,0.1094,17.8325,2.1704,2.1111,0.8466,6.8530e-03,0.0187,76.7896,16.6400
...,...,...,...,...,...,...,

In [70]:
y = trn['class'].values
trn_df = trn.drop('class', axis = 1).values
tst_df = tst.values

trn_df.shape, y.shape, tst_df.shape

((318740, 90), (318740,), (80000, 90))

In [14]:
features = trn.columns
n_fold = 10

In [92]:
def objective(trial ,rounds):
  n_class = 3
  on_gpu=1
  p_val = np.zeros((trn_df.shape[0], n_class))
  cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
  
  for i, (i_trn, i_val) in enumerate(cv.split(trn_df, y), 1):
    param = {"objective": "multiclass",
           'learning_rate' : trial.suggest_loguniform('learning_rate',1e-3,1e-2),
           'num_iterations':trial.suggest_int('num_iterations',100,1000),           
           'n_estimators': trial.suggest_int('n_estimators',5000,20000),
           'num_leaves': trial.suggest_int('num_leaves', 31, 512),
           'max_depth': trial.suggest_int('max_depth',12,27),
           'boosting_type':'gbdt',
           'metric':  'multi_logloss',
           'num_class': n_class,
           'verbose':-1}

    if on_gpu==1:
      print('Use_GPU')
      gpu={'device': 'gpu'}
      param.update(gpu)
    print(param)

    dtrain = lgb.Dataset(trn_df[i_trn], label=y[i_trn])
    dvalid = lgb.Dataset(trn_df[i_val], label=y[i_val])
    print(f'training model for CV #{i}')
  # pruning_callback = optuna.integration.LightGBMPruningCallback(trial, '')
    clf = lgb.train(param,
                    dtrain,
                    valid_sets=[dtrain,dvalid],
                    early_stopping_rounds=rounds,
  #                  callbacks=[pruning_callback],
                    verbose_eval = 100
                    )
     
    p_val[i_val, :] = clf.predict(trn_df[i_val])
    loss = log_loss(y[i_val],p_val[i_val])
    
    print('best_score', clf.best_score)
    print(f'multi logloss: {loss}')

  # print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')
  return loss

In [ ]:
study0 = optuna.create_study(pruner=optuna.pruners.MedianPruner(n_warmup_steps=5))
study0.optimize(lambda trial: objective(trial,1), n_trials=10)

[I 2020-10-04 05:51:25,087] A new study created in memory with name: no-name-88acc8c8-3567-483f-9a0c-9010bcb814aa


Use_GPU
{'objective': 'multiclass', 'learning_rate': 0.0013359532471429157, 'num_iterations': 803, 'n_estimators': 9833, 'num_leaves': 372, 'max_depth': 17, 'boosting_type': 'gbdt', 'metric': 'multi_logloss', 'num_class': 3, 'verbose': -1, 'device': 'gpu'}
training model for CV #1
Training until validation scores don't improve for 1 rounds


In [ ]:
print(study0.best_trial.params)